In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf

In [2]:
landmarks_list = {
    "left_shoulder": [], "right_shoulder": [],
    "left_elbow": [], "right_elbow": [],
    "left_wrist": [], "right_wrist": [],
    "left_hip": [], "right_hip": [],
    "left_knee": [], "right_knee": [],
    "left_ankle": [], "right_ankle": [],
}

In [3]:
def angle(p1, p2, p3):
    a = np.array([p1[0], p1[1]])
    b = np.array([p2[0], p2[1]])
    c = np.array([p3[0], p3[1]])

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
        np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180:
        angle = 360 - angle

    return angle

In [ ]:
interpreter = tf.lite.Interpreter(model_path="./models/t_3.tflite")
interpreter.allocate_tensors()

In [5]:
landmarks = {
    "nose": 0,
    'left_shoulder': 5, 'right_shoulder': 6,
    'left_elbow': 7, 'right_elbow': 8,
    'left_wrist': 9, 'right_wrist': 10,
    'left_hip': 11, 'right_hip': 12,
    'left_knee': 13, 'right_knee': 14,
    'left_ankle': 15, 'right_ankle': 16,
}

col_names = []
for i in range(len(landmarks.keys())):
    name = list(landmarks.keys())[i]
    col_names.append(name + "_x")
    col_names.append(name + "_y")
    col_names.append(name + "_confidence")

pose_name = col_names.copy()

pose_name.append("pose")

In [6]:
main_dir = "./poses_dataset/Dataset"
pose_dir_list = os.listdir(main_dir)
pose_list = []
file = open("angles.csv", "w")
file.write(
    "class,armpit_left,armpit_right,elbow_left,elbow_right,hip_left,hip_right,knee_left,knee_right\n")

for i in range(0, len(pose_dir_list)):
    images_dir_list = os.listdir(f"{main_dir}/{pose_dir_list[i]}")
    for l in range(0, len(images_dir_list)):
        pre_list = []

        image = cv2.imread(f"{main_dir}/{pose_dir_list[i]}/{images_dir_list[l]}")
        print(pose_dir_list[i])
        print(images_dir_list[l])
        # Convert image to required format
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, (256, 256))  # MoveNet requires 256x256 input
        img_expanded = np.expand_dims(img_resized.astype(np.float32), axis=0)
        
        # Setup input and output
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        # Make predictions
        interpreter.set_tensor(input_details[0]['index'], img_expanded)
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])[0, 0]

        try:
            for keypoint in keypoints_with_scores:
                x, y, score = keypoint[:3]  # Extract only x, y, and confidence score
                pre_list.append((x, y, score))
            predict = True
        except AttributeError:
            print(f"No points {main_dir}/{pose_dir_list[i]}/{images_dir_list[l]}")
            predict = False


        if predict == True:
            gen0510 = np.array([
                [pre_list[m][0], pre_list[m][1], pre_list[m][2]] for m in range(5, 11)
            ]).flatten().tolist()

            gen1116 = np.array([
                [pre_list[m][0], pre_list[m][1], pre_list[m][2]] for m in range(11, 17)
            ]).flatten().tolist()

            gen0510.extend(gen1116)

            all_list = [pre_list[0][0], pre_list[0][1], pre_list[0][2]]
            all_list.extend(gen0510)

            tpl = all_list.copy()
            tpl.append(i)
            pose_list.append(tpl)
print(pose_list)
print(pose_name)
data_pose = pd.DataFrame(pose_list, columns=pose_name)

for i, row in data_pose.iterrows():
    sl = []
    landmarks_list["left_shoulder"] = [
        row["left_shoulder_x"], row["left_shoulder_y"]]

    landmarks_list["right_shoulder"] = [
        row["right_shoulder_x"], row["right_shoulder_y"]]

    landmarks_list["left_elbow"] = [
        row["left_elbow_x"], row["left_elbow_y"]]

    landmarks_list["right_elbow"] = [
        row["right_elbow_x"], row["right_elbow_y"]]

    landmarks_list["left_wrist"] = [
        row["left_wrist_x"], row["left_wrist_y"]]

    landmarks_list["right_wrist"] = [
        row["right_wrist_x"], row["right_wrist_y"]]

    landmarks_list["left_hip"] = [
        row["left_hip_x"], row["left_hip_y"]]

    landmarks_list["right_hip"] = [
        row["right_hip_x"], row["right_hip_y"]]

    landmarks_list["left_knee"] = [
        row["left_knee_x"], row["left_knee_y"]]

    landmarks_list["right_knee"] = [
        row["right_knee_x"], row["right_knee_y"]]

    landmarks_list["left_ankle"] = [
        row["left_ankle_x"], row["left_ankle_y"]]

    landmarks_list["right_ankle"] = [
        row["right_ankle_x"], row["right_ankle_y"]]

    armpit_left = angle(
        landmarks_list["left_elbow"],
        landmarks_list["left_shoulder"],
        landmarks_list["left_hip"]
    )
    armpit_right = angle(
        landmarks_list["right_elbow"],
        landmarks_list["right_shoulder"],
        landmarks_list["right_hip"]
    )

    elbow_left = angle(
        landmarks_list["left_shoulder"],
        landmarks_list["left_elbow"],
        landmarks_list["left_wrist"]
    )
    elbow_right = angle(
        landmarks_list["right_shoulder"],
        landmarks_list["right_elbow"],
        landmarks_list["right_wrist"]
    )

    hip_left = angle(
        landmarks_list["right_hip"],
        landmarks_list["left_hip"],
        landmarks_list["left_knee"]
    )
    hip_right = angle(
        landmarks_list["left_hip"],
        landmarks_list["right_hip"],
        landmarks_list["right_knee"]
    )

    knee_left = angle(
        landmarks_list["left_hip"],
        landmarks_list["left_knee"],
        landmarks_list["left_ankle"]
    )
    knee_right = angle(
        landmarks_list["right_hip"],
        landmarks_list["right_knee"],
        landmarks_list["right_ankle"]
    )

    tmp = [
        armpit_left,
        armpit_right,
        elbow_left,
        elbow_right,
        hip_left,
        hip_right,
        knee_left,
        knee_right,
    ]

    sl.append(tmp)
    file.write(
        f"{int(row['pose'])},{','.join(map(lambda x: str(int(round(x))), sl[0]))}\n")
    print(f"{int(row['pose'])},{','.join(map(lambda x: str(int(round(x))), sl[0]))}")
file.close()

Adho Mukha Svanasana
File1.png
Adho Mukha Svanasana
File10.png
Adho Mukha Svanasana
File13.png
Adho Mukha Svanasana
File14.png
Adho Mukha Svanasana
File15.png
Adho Mukha Svanasana
File16.png
Adho Mukha Svanasana
File17.png
Adho Mukha Svanasana
File19.png
Adho Mukha Svanasana
File2.png
Adho Mukha Svanasana
File21.png
Adho Mukha Svanasana
File22.png
Adho Mukha Svanasana
File23.png
Adho Mukha Svanasana
File25.jpg
Adho Mukha Svanasana
File26.png
Adho Mukha Svanasana
File27.png
Adho Mukha Svanasana
File28.png
Adho Mukha Svanasana
File29.png
Adho Mukha Svanasana
File30.png
Adho Mukha Svanasana
File31.png
Adho Mukha Svanasana
File32.png
Adho Mukha Svanasana
File33.png
Adho Mukha Svanasana
File34.png
Adho Mukha Svanasana
File35.png
Adho Mukha Svanasana
File36.png
Adho Mukha Svanasana
File37.png
Adho Mukha Svanasana
File38.png
Adho Mukha Svanasana
File39.png
Adho Mukha Svanasana
File4.png
Adho Mukha Svanasana
File40.png
Adho Mukha Svanasana
File41.png
Adho Mukha Svanasana
File42.png
Adho Mukha 

In [15]:
import pandas as pd

# Load the CSV file with individual pose data
df = pd.read_csv("angles.csv")  # Make sure this is the correct filename

# Compute the mean values for each class
df_avg = df.groupby("class").mean().reset_index()

# Save the results to a new CSV file
df_avg.to_csv("angles_classwise_avg.csv", index=False)

print("Averaged data saved to angles_classwise_avg.csv")


Averaged data saved to angles_classwise_avg.csv
